In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import morphology
from skimage.measure import label

In [2]:
def neighbours4(y, x):
  return (y,x-1),(y,x+1),(y-1,x),(y+1,x)


def get_boundaries(LB,label):
  pxs = np.where(LB == label)
  boundaries = []
  for y,x in zip(*pxs):
    for yn, xn in neighbours4(y,x):
      if yn < 0 or yn > LB.shape[0] - 1:
        boundaries.append((y,x))
        break
      elif xn < 0 or xn > LB.shape[1] - 1:
        boundaries.append((y,x))
        break
      elif LB[yn,xn] != label:
        boundaries.append((y,x))    
        break
  return boundaries

def draw_boundaries(LB,label):
  BB = np.zeros_like(LB)
  pos = np.where(LB == label)
  BB[pos] = LB[pos]

  boundaries = get_boundaries(BB,label)
  for y, x in boundaries:
    BB[y,x] = label + 1
  return BB

In [41]:
file = np.load('ps.npy')

In [40]:
labeled = label(file)

In [39]:
print(np.max(labeled))

500


In [6]:
def get_boundary_path(image,label):
  boundaries = get_boundaries(image,label)
  moves = [(-1,0),(-1,1),(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1)]
  path = []
  coords = boundaries[0]
  while len(boundaries) > 0:
    move_candidates = []
    new_coords = ()
    for move in moves:
      new_coords = (coords[0]+move[0],coords[1]+move[1])
      if new_coords in boundaries:
        move_candidates.append(True)
      else:
        move_candidates.append(False)
    
    if not move_candidates[-1] and move_candidates[0]:
      coords = (coords[0]+moves[0][0],coords[1]+moves[0][1])
      path.append(0)
    else:
      for i in range(7):
        if not move_candidates[i] and move_candidates[i+1]:
          coords = (coords[0]+moves[i+1][0],coords[1]+moves[i+1][1])
          path.append(i+1)
          break

    boundaries.remove(coords)
  return sorted(path)

In [29]:
image = file.copy()
masks = []
total = np.max(labeled)
found = 0
types = {}
for i in range(1,total+1):
  pxs = np.where(labeled == i)
  if len(pxs[0])>0:
    labeled_n = np.zeros_like(labeled)
    labeled_n[pxs] = 1
    cropped = labeled_n[min(pxs[0]):max(pxs[0])+1,min(pxs[1]):max(pxs[1])+1]
    boundary_path = np.array(get_boundary_path(cropped,1))
    new_type = "".join([str(e) for e in boundary_path])
    if new_type in types:
      continue
    else:
      result = morphology.binary_opening(image,cropped).astype('int')
      labeled_c = label(result)
      amount = np.max(labeled_c)
      types[new_type] = amount
      found += amount

In [43]:
r_types = {}
for t in types:
  path = np.array([int(i) for i in t])
  paths = [sorted((path + i)%8) for i in range(0,7,2)]
  paths = ''.join(sorted([''.join([str(ch) for ch in p]) for p in paths]))
  
  if paths in r_types:
    r_types[paths]+=types[t]
  else:
    r_types[paths] = types[t]
r_types = [t for t in r_types.values()]

print('Total:',np.max(labeled),'First type:',r_types[0]-r_types[1]*2,'Second type:',r_types[1])

Total: 500 First type: 408 Second type: 92
